In [12]:
from pandas import DataFrame, read_csv, concat
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, GRU,ConvLSTM2D, Flatten
from matplotlib import pyplot as plt
from numpy import concatenate, reshape, array, squeeze
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sys import argv
import csv
import datetime
import time
import pickle
from keras.models import model_from_json
import talib

In [25]:
#Set Feature Combinations
def get_CurrentCombo(dataframe, combo_number):
    # Add features
    sma = talib.SMA(dataframe['open '+fileName], timeperiod = 10)
    ema = talib.EMA(dataframe['open '+fileName], timeperiod = 10)
    macd = talib.MACD(dataframe['open '+fileName])[0]
    rsi = talib.RSI(dataframe['open '+fileName])
    mom = talib.MOM(dataframe['open '+fileName], timeperiod=15)

    dataframe.insert(5,'sma',sma)
    dataframe.insert(6,'ema',ema)
    dataframe.insert(7,'macd',macd)
    dataframe.insert(8,'rsi',rsi)
    dataframe.insert(9,'mom',mom)

    combinations = [[5,6,7,8,9],[4,5,6],[3,4,5,6],[3,4,7,9,8],[4,7,9,8]]
    newDataFrame = dataframe.drop(dataframe.columns[combinations[combo_number]], axis = 1)
    return newDataFrame

#Load the training dataset
fileName = 'BTC' #asset name
dataframe = read_csv('final_datasets/'+fileName+'.csv', parse_dates=['time']) 
startIndex = 3  #start from 3rd column
nrows = dataframe.shape[0]


passDataframe = dataframe

In [26]:
feature_combo=3#0,1,2,3,4 for different feature combinations

dataframe = get_CurrentCombo(passDataframe, feature_combo)

In [27]:
Colname= dataframe.columns[3:-1]
Colname = '-'.join(Colname)


modelFeature = Colname #FeatureNames
json_file = open('models_rbscaler/{}/{}_{}_model.json'.format(fileName,fileName, modelFeature), 'r') #Load model
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights('models_rbscaler/{}/{}_{}_model.h5'.format(fileName,fileName, modelFeature))
print("Loaded model from disk")


dataset=dataframe.iloc[33:,:]



values = dataset.iloc[:,startIndex:].values #Getting values - Total Sentiment and BTC Values
valuesCrypto = dataset.iloc[:,-1:].values #Getting values -  Crypto Values
dataset

Loaded model from disk


,time,sentiment_score,Tweet Volume,sma,ema,open BTC
33,2019-01-05 03:00:00,-0.163479,2741,3795.491000,3807.406862,3844.130000
34,2019-01-05 06:00:00,-0.162995,2991,3801.462667,3813.005009,3838.196667
35,2019-01-05 09:00:00,-0.095735,3236,3805.302333,3815.998643,3829.470000
36,2019-01-05 12:00:00,-0.106470,3687,3808.631333,3819.372223,3834.553333
37,2019-01-05 15:00:00,-0.104451,4370,3812.785000,3821.488789,3831.013333
...,...,...,...,...,...,...
7413,2021-07-17 03:00:00,-0.010676,18590,31691.137667,31693.380903,31350.700000
7414,2021-07-17 06:00:00,-0.016053,19695,31666.760667,31664.125588,31532.476670
7415,2021-07-17 09:00:00,-0.007262,20159,31629.330334,31641.746390,31541.040000
7416,2021-07-17 12:00:00,-0.041026,26502,31607.032667,31618.474925,31513.753330


In [28]:
print('Features:', Colname)

Features: sma-ema


In [68]:
#load test dataset 

test_dataframe= read_csv('extendedDatasets/'+fileName+'.csv', parse_dates=['time'])
passtest_dataframe=test_dataframe
# test_dataframe = get_CurrentCombo(passtest_dataframe, feature_combo)

# Existing dataframe(atleast 36 past values)
test_dataset=test_dataframe.iloc[:100,:].reset_index(drop=True)

# Simulating real time data (new data that is received every three hours)
realTimeData=test_dataframe.iloc[100:].reset_index(drop=True)

In [69]:
#Robust Scaler
rb_tranformer = RobustScaler().fit(values)
rb_transformed=rb_tranformer.fit_transform(values)
rb_Crypto= RobustScaler().fit(valuesCrypto)
rb_trans_Crypto=rb_Crypto.fit_transform(valuesCrypto)

# MinMax Scaler
scaler = MinMaxScaler(feature_range = (0,1))
scaler = scaler.fit(rb_transformed)
scaled = scaler.fit_transform(rb_transformed)


In [70]:
step=3
current_DF = test_dataset

for index, row in realTimeData.iterrows():
    
    # Row in real time environment can be fetched from API
    print(index, row)
    
    # Appending new data to existing dataframe
    current_DF=current_DF.append(row)
    saved_current_DF = current_DF.copy()
    saved_current_DF = get_CurrentCombo(saved_current_DF, feature_combo)
    
    # Since the model requires n-timesteps as input sequence, we take last n steps to make prediction 
    current_test_DF = saved_current_DF.iloc[-step:,startIndex:].values
    print("Before Scaling: ",current_test_DF)
    
    # Scaling
    scaled_rb = rb_tranformer.transform(current_test_DF)
    scaled_test=scaler.transform(scaled_rb)
    test_X = scaled_test.reshape((1,step, scaled_test.shape[1]))
    print("The values scaled")
    print(test_X)
    
    # Load model and predict
    
    model = loaded_model
    model_prediction = model.predict(test_X)
    
    # Inverse Scale
    scalerCrypto = MinMaxScaler(feature_range = (0,1))
    scalerCrypto = scalerCrypto.fit(rb_trans_Crypto)
    scaledCrypto = scalerCrypto.inverse_transform(model_prediction)

    rb_tranformer_Crypto = RobustScaler().fit(valuesCrypto)
    rb_transformedCrypto=rb_tranformer_Crypto.fit_transform(valuesCrypto)
    unscaled_Crypto = rb_tranformer_Crypto.inverse_transform(scaledCrypto)
    
    model_prediction_unscale = unscaled_Crypto
    predictedValues = reshape(model_prediction_unscale, model_prediction_unscale.shape[0])
    print('Predicted Value: ',predictedValues)
    
    #Generate buy/sell signals
    
    #Sleep for 3 hours
    
    break

0 time                  2021-06-13 12:00:00
sentiment_score                  0.044851
tweet volume                        29984
Weighted Sentiment            1344.818102
Volume BTC                     929.652547
open BTC                     35902.176667
Name: 0, dtype: object
Before Scaling:  [[ 2019.92027838  -170.82117504    41.52253406 -1985.59666667
  35214.81      ]
 [  967.04554611  -153.35459304    48.82989327 -1575.15666667
  35821.44      ]
 [ 1344.81810168  -131.48178652    49.73024111 -1009.18666667
  35902.17666667]]
The values scaled
[[[0.34081746 0.4765414  0.39182856 0.46107116 0.52225927]
  [0.31040787 0.48029789 0.47217791 0.48201303 0.53220436]
  [0.32131886 0.48500201 0.48207784 0.51089052 0.53352796]]]
Predicted Value:  [35975.445]
1 time                  2021-06-13 15:00:00
sentiment_score                   0.01043
tweet volume                        37700
Weighted Sentiment             393.195598
Volume BTC                    2514.286657
open BTC                  

In [ ]:
# Alternative code that might not work in production

# step=3

# for index, row in test_dataset.iterrows():
#     print(index, row)
    
#     '''
#     This is where the scaling will happen during production 
#     Assuming that the test dataset keeps updating with new values every 3 hours,
#     we can change the code to pick the last row of the test dataset to make the next prediction
#     '''
#     test_values = test_dataset.iloc[index:index+step, startIndex:].values 
    
#     print("Last 3 value required to make prediction")
#     print(test_dataset.iloc[index:index+step])
#     print("----------")
#     print(test_values)
    

#     # Scaling the test_values according to the min-0 and max-1 from the training dataset
#     scaled_rb = rb_tranformer.transform(test_values)
#     scaled_test=scaler.transform(scaled_rb)
#     print("The values scaled")
#     print(scaled_test)
    
#     #Reshaping input to be 3 dimensions (samples, timesteps, features)

#     test_X = scaled_test.reshape((1,step, scaled_test.shape[1]))
    
#     # Predicition using the loaded model
#     model = loaded_model
#     model_prediction = model.predict(test_X)
    
#     # Inverse Scale
    
    
#     scalerCrypto = MinMaxScaler(feature_range = (0,1))
#     scalerCrypto = scalerCrypto.fit(rb_trans_Crypto)
#     scaledCrypto = scalerCrypto.inverse_transform(model_prediction)

#     rb_tranformer_Crypto = RobustScaler().fit(valuesCrypto)
#     rb_transformedCrypto=rb_tranformer_Crypto.fit_transform(valuesCrypto)
#     unscaled_Crypto = rb_tranformer_Crypto.inverse_transform(scaledCrypto)
    
#     model_prediction_unscale = unscaled_Crypto
#     predictedValues = reshape(model_prediction_unscale, model_prediction_unscale.shape[0])
#     print('Predicted Value: ',predictedValues)
    
#     #Generate buy/sell signals
    
#     #Sleep for 3 hours
    
#     break